In [ ]:
# 설치
! pip install smolagents
# 마지막 릴리스 대신 소스에서 설치하려면 위 명령을 주석 처리하고 다음 명령의 주석을 해제하십시오.
# ! pip install git+https://github.com/huggingface/smolagents.git

# 좋은 에이전트 구축하기

작동하는 에이전트와 그렇지 않은 에이전트를 구축하는 것 사이에는 엄청난 차이가 있습니다.
어떻게 하면 전자에 해당하는 에이전트를 구축할 수 있을까요?
이 가이드에서는 에이전트 구축을 위한 모범 사례에 대해 이야기합니다.

> [!TIP]
> 에이전트 구축이 처음이라면 먼저 [에이전트 소개](https://huggingface.co/docs/smolagents/main/ko/tutorials/../conceptual_guides/intro_agents) 및 [smolagents 둘러보기](https://huggingface.co/docs/smolagents/main/ko/tutorials/../guided_tour)를 읽어보십시오.

### 최고의 에이전트 시스템은 가장 단순합니다: 워크플로를 최대한 단순화하십시오.

워크플로에 LLM 에이전시를 부여하면 오류 위험이 발생합니다.

잘 프로그래밍된 에이전트 시스템에는 어쨌든 좋은 오류 로깅 및 재시도 메커니즘이 있으므로 LLM 엔진은 실수를 스스로 수정할 기회가 있습니다. 그러나 LLM 오류 위험을 최대한 줄이려면 워크플로를 단순화해야 합니다!

 [에이전트 소개](https://huggingface.co/docs/smolagents/main/ko/tutorials/../conceptual_guides/intro_agents)의 예로 돌아가 보겠습니다: 서핑 여행 회사를 위한 사용자 문의에 답변하는 봇입니다.
새로운 서핑 장소에 대해 질문을 받을 때마다 에이전트가 "여행 거리 API"와 "날씨 API"에 대해 두 번의 다른 호출을 하도록 하는 대신, 두 API를 한 번에 호출하고 연결된 출력을 사용자에게 반환하는 함수인 "return_spot_information"이라는 통합 도구를 만들 수 있습니다.

이렇게 하면 비용, 지연 시간 및 오류 위험이 줄어듭니다!

주요 지침은 다음과 같습니다: LLM 호출 횟수를 최대한 줄이십시오.

이는 몇 가지 중요한 점으로 이어집니다:
- 가능하면 두 API의 예에서처럼 두 개의 도구를 하나로 그룹화하십시오.
- 가능하면 에이전트 결정보다는 결정적 함수를 기반으로 논리를 만들어야 합니다.

### LLM 엔진으로의 정보 흐름 개선

LLM 엔진은 문 아래로 전달되는 메모만으로 외부 세계와 소통하는 방에 갇힌 *지능적인* 로봇과 같다는 것을 기억하십시오.

프롬프트에 명시적으로 입력하지 않으면 어떤 일이 일어났는지 알 수 없습니다.

따라서 먼저 작업을 매우 명확하게 만드는 것부터 시작하십시오!
에이전트는 LLM으로 구동되므로 작업 공식화의 사소한 변형으로 인해 완전히 다른 결과가 나올 수 있습니다.

그런 다음 도구 사용에서 에이전트로의 정보 흐름을 개선하십시오.

따라야 할 특정 지침:
- 각 도구는 도구의 `forward` 메서드 내에서 `print` 문을 사용하여 LLM 엔진에 유용할 수 있는 모든 것을 기록해야 합니다.
  - 특히 도구 실행 오류에 대한 세부 정보를 기록하면 많은 도움이 됩니다!

예를 들어, 다음은 위치 및 날짜-시간을 기반으로 날씨 데이터를 검색하는 도구입니다:

먼저, 잘못된 버전입니다:

In [ ]:
import datetime
from smolagents import tool

def get_weather_report_at_coordinates(coordinates, date_time):
    # 더미 함수, [섭씨 온도, 강수 확률(0-1), 파고(m)] 목록을 반환합니다.
    return [28.0, 0.35, 0.85]

def convert_location_to_coordinates(location):
    # 더미 좌표를 반환합니다.
    return [3.3, -42.0]

@tool
def get_weather_api(location: str, date_time: str) -> str:
    """
    날씨 예보를 반환합니다.

    Args:
        location: 날씨를 원하는 장소의 이름입니다.
        date_time: 예보를 원하는 날짜 및 시간입니다.
    """
    lon, lat = convert_location_to_coordinates(location)
    date_time = datetime.strptime(date_time)
    return str(get_weather_report_at_coordinates((lon, lat), date_time))

왜 나쁜가요?
- `date_time`에 사용해야 하는 형식에 대한 정밀도가 없습니다.
- 위치를 지정하는 방법에 대한 세부 정보가 없습니다.
- 위치가 적절한 형식이 아니거나 date_time이 제대로 형식이 지정되지 않은 경우와 같은 명시적인 실패 사례를 만들려는 로깅 메커니즘이 없습니다.
- 출력 형식을 이해하기 어렵습니다.

도구 호출이 실패하면 메모리에 기록된 오류 추적은 LLM이 오류를 수정하기 위해 도구를 리버스 엔지니어링하는 데 도움이 될 수 있습니다. 그러나 왜 그렇게 많은 힘든 작업을 하도록 내버려 두나요?

이 도구를 구축하는 더 좋은 방법은 다음과 같습니다:

In [ ]:
@tool
def get_weather_api(location: str, date_time: str) -> str:
    """
    날씨 예보를 반환합니다.

    Args:
        location: 날씨를 원하는 장소의 이름입니다. "Anchor Point, Taghazout, Morocco"와 같이 장소 이름, 도시 이름, 국가 순으로 지정해야 합니다.
        date_time: 예보를 원하는 날짜 및 시간이며 '%m/%d/%y %H:%M:%S' 형식으로 지정합니다.
    """
    lon, lat = convert_location_to_coordinates(location)
    try:
        date_time = datetime.strptime(date_time)
    except Exception as e:
        raise ValueError("`date_time`을 datetime 형식으로 변환하지 못했습니다. '%m/%d/%y %H:%M:%S' 형식의 문자열을 제공했는지 확인하십시오. 전체 추적:" + str(e))
    temperature_celsius, risk_of_rain, wave_height = get_weather_report_at_coordinates((lon, lat), date_time)
    return f"{location}, {date_time}의 날씨 예보: 온도는 {temperature_celsius}°C, 강수 확률은 {risk_of_rain*100:.0f}%, 파고는 {wave_height}m입니다."

일반적으로 LLM의 부하를 줄이려면 스스로에게 다음과 같은 좋은 질문을 던져야 합니다. "내가 바보이고 이 도구를 처음 사용하는 경우 이 도구로 프로그래밍하고 내 오류를 수정하는 것이 얼마나 쉬울까요?"

### 에이전트에 더 많은 인수 제공

작업을 설명하는 간단한 문자열 외에 에이전트에 추가 개체를 전달하려면 `additional_args` 인수를 사용하여 모든 유형의 개체를 전달할 수 있습니다.

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

model_id = "meta-llama/Llama-3.3-70B-Instruct"

agent = CodeAgent(tools=[], model=InferenceClientModel(model_id=model_id), add_base_tools=True)

agent.run(
    "마이크는 왜 뉴욕에 아는 사람이 많지 않나요?",
    additional_args={"mp3_sound_file_url":'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/recording.mp3'}
)

예를 들어, 이 `additional_args` 인수를 사용하여 에이전트가 활용하기를 원하는 이미지나 문자열을 전달할 수 있습니다.

## 에이전트 디버깅 방법

### 1. 더 강력한 LLM 사용

에이전트 워크플로에서 일부 오류는 실제 오류이고 다른 일부는 LLM 엔진이 제대로 추론하지 못한 탓입니다.
예를 들어, 자동차 사진을 만들도록 요청한 `CodeAgent`에 대한 다음 추적을 고려하십시오:
```
==================================================================================================== 새 작업 ====================================================================================================
멋진 자동차 사진을 만들어 주세요.
──────────────────────────────────────────────────────────────────────────────────────────────────── 새 단계 ────────────────────────────────────────────────────────────────────────────────────────────────────
에이전트가 아래 코드를 실행 중입니다: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
image_generator(prompt="LED 헤드라이트, 공기역학적 디자인, 생생한 색상의 멋지고 미래 지향적인 스포츠카, 고해상도, 사실적")
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

코드 스니펫의 마지막 출력: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/tmpx09qfsdd/652f0007-3ee9-44e2-94ac-90dae6bb89a4.png
1단계:

- 소요 시간: 16.35초
- 입력 토큰: 1,383
- 출력 토큰: 77
──────────────────────────────────────────────────────────────────────────────────────────────────── 새 단계 ────────────────────────────────────────────────────────────────────────────────────────────────────
에이전트가 아래 코드를 실행 중입니다: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
final_answer("/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/tmpx09qfsdd/652f0007-3ee9-44e2-94ac-90dae6bb89a4.png")
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
출력 인쇄:

코드 스니펫의 마지막 출력: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/tmpx09qfsdd/652f0007-3ee9-44e2-94ac-90dae6bb89a4.png
최종 답변:
/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/tmpx09qfsdd/652f0007-3ee9-44e2-94ac-90dae6bb89a4.png
```
사용자는 반환된 이미지가 아닌 경로가 반환되는 것을 봅니다.
시스템의 버그처럼 보일 수 있지만 실제로는 에이전트 시스템이 오류를 일으킨 것이 아닙니다. LLM 두뇌가 이미지 출력을 변수에 저장하지 않는 실수를 저질렀기 때문입니다.
따라서 이미지를 저장하는 동안 기록된 경로를 활용하는 것 외에는 이미지에 다시 액세스할 수 없으므로 이미지 대신 경로를 반환합니다.

따라서 에이전트 디버깅의 첫 번째 단계는 "더 강력한 LLM 사용"입니다. `Qwen2/5-72B-Instruct`와 같은 대안은 그러한 실수를 저지르지 않았을 것입니다.

### 2. 더 많은 정보 또는 특정 지침 제공

더 효과적으로 안내한다면 덜 강력한 모델을 사용할 수도 있습니다.

모델의 입장이 되어 보십시오. 작업을 해결하는 모델이라면 사용 가능한 정보(시스템 프롬프트 + 작업 공식화 + 도구 설명)로 어려움을 겪을까요?

자세한 지침이 필요할까요?

- 지침이 항상 에이전트에게 주어져야 하는 경우(일반적으로 시스템 프롬프트가 작동하는 방식): 에이전트 초기화 시 `instructions` 인수 아래에 문자열로 전달할 수 있습니다.
- 해결해야 할 특정 작업에 관한 경우: 이러한 모든 세부 정보를 작업에 추가하십시오. 작업은 수십 페이지처럼 매우 길 수 있습니다.
- 특정 도구를 사용하는 방법에 관한 경우: 이러한 도구의 `description` 속성에 포함하십시오.

### 3. 프롬프트 템플릿 변경 (일반적으로 권장되지 않음)

위의 설명이 충분하지 않으면 에이전트의 프롬프트 템플릿을 변경할 수 있습니다.

어떻게 작동하는지 봅시다. 예를 들어, [CodeAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.CodeAgent)의 기본 프롬프트 템플릿을 확인해 봅시다(아래 버전은 제로샷 예제를 건너뛰어 단축됨).

In [ ]:
print(agent.prompt_templates["system_prompt"])

다음과 같은 결과를 얻습니다:
```text
당신은 코드 블록을 사용하여 모든 작업을 해결할 수 있는 전문가 조수입니다. 최선을 다해 해결해야 할 작업이 주어집니다.
그렇게 하기 위해 도구 목록에 액세스할 수 있습니다. 이러한 도구는 기본적으로 코드로 호출할 수 있는 Python 함수입니다.
작업을 해결하려면 '생각:', '코드:', '관찰:' 시퀀스의 주기로 일련의 단계로 진행하도록 계획해야 합니다.

각 단계에서 '생각:' 시퀀스에서는 먼저 작업 해결을 위한 추론과 사용하려는 도구를 설명해야 합니다.
그런 다음 '코드:' 시퀀스에서는 간단한 Python으로 코드를 작성해야 합니다. 코드 시퀀스는 '<end_code>' 시퀀스로 끝나야 합니다.
각 중간 단계에서 'print()'를 사용하여 필요한 중요한 정보를 저장할 수 있습니다.
이러한 인쇄 출력은 다음 단계의 입력으로 사용할 수 있는 '관찰:' 필드에 나타납니다.
결국 `final_answer` 도구를 사용하여 최종 답변을 반환해야 합니다.

다음은 가상 도구를 사용하는 몇 가지 예입니다:
---
작업: "이 문서에서 가장 나이 많은 사람의 이미지를 생성하십시오."

생각: 단계별로 진행하고 다음 도구를 사용합니다: 문서에서 가장 나이 많은 사람을 찾기 위한 `document_qa`, 그런 다음 답변에 따라 이미지를 생성하기 위한 `image_generator`.
<code>
answer = document_qa(document=document, question="언급된 가장 나이 많은 사람은 누구입니까?")
print(answer)
</code>
관찰: "문서에서 가장 나이 많은 사람은 뉴펀들랜드에 사는 55세 벌목꾼인 존 도입니다."

생각: 이제 가장 나이 많은 사람을 보여주는 이미지를 생성합니다.
<code>
image = image_generator("캐나다에 사는 55세 남성 존 도의 초상화.")
final_answer(image)
</code>

---
작업: "다음 연산의 결과는 무엇입니까: 5 + 3 + 1294.678?"

생각: 파이썬 코드를 사용하여 연산 결과를 계산한 다음 `final_answer` 도구를 사용하여 최종 답변을 반환합니다.
<code>
result = 5 + 3 + 1294.678
final_answer(result)
</code>

---
작업:
"변수 `image`에 저장된 이미지에 대한 변수 `question`의 질문에 답하십시오. 질문은 프랑스어입니다.
파이썬 코드에서 키를 변수로 사용하여 액세스할 수 있는 다음과 같은 추가 인수가 제공되었습니다:
{'question': '이미지에 있는 동물은 무엇입니까?', 'image': 'path/to/image.jpg'}"

생각: 다음 도구를 사용합니다: 질문을 영어로 번역하기 위한 `translator`, 그런 다음 입력 이미지에 대한 질문에 답하기 위한 `image_qa`.
<code>
translated_question = translator(question=question, src_lang="French", tgt_lang="English")
print(f"번역된 질문은 {translated_question}입니다.")
answer = image_qa(image=image, question=translated_question)
final_answer(f"답변은 {answer}입니다.")
</code>

---
작업:
1979년 인터뷰에서 스타니스와프 울람은 마틴 셔윈과 함께 오펜하이머를 포함한 당대의 다른 위대한 물리학자들에 대해 논의합니다.
아인슈타인이 수학을 너무 많이 배운 것이 그의 창의성에 어떤 결과를 가져왔다고 그는 한마디로 말합니까?

생각: 마틴 셔윈과의 스타니스와프 울람의 1979년 인터뷰를 찾아 읽어야 합니다.
코드:

In [ ]:
pages = search(query="1979 interview Stanislaus Ulam Martin Sherwin physicists Einstein")
print(pages)
```<end_code>
관찰:
쿼리 "1979 interview Stanislaus Ulam Martin Sherwin physicists Einstein"에 대한 결과가 없습니다.

생각: 쿼리가 너무 제한적이어서 결과를 찾지 못했을 수 있습니다. 더 넓은 쿼리로 다시 시도해 봅시다.
코드:
pages = search(query="1979 interview Stanislaus Ulam")
print(pages)
```<end_code>
관찰:
6페이지를 찾았습니다:
[스타니스와프 울람 1979년 인터뷰](https://ahf.nuclearmuseum.org/voices/oral-histories/stanislaus-ulams-interview-1979/)

[울람, 맨해튼 프로젝트 논의](https://ahf.nuclearmuseum.org/manhattan-project/ulam-manhattan-project/)

(생략)

생각: 자세히 알기 위해 처음 2페이지를 읽겠습니다.
코드:
for url in ["https://ahf.nuclearmuseum.org/voices/oral-histories/stanislaus-ulams-interview-1979/", "https://ahf.nuclearmuseum.org/manhattan-project/ulam-manhattan-project/"]:
    whole_page = visit_webpage(url)
    print(whole_page)
    print("\n" + "="*80 + "\n")  # 페이지 사이에 구분 기호 인쇄
```<end_code>
관찰:
맨해튼 프로젝트 위치:
로스앨러모스, NM
스타니스와프 울람은 폴란드계 미국인 수학자였습니다. 그는 로스앨러모스에서 맨해튼 프로젝트에 참여했으며 나중에 수소 폭탄 설계에 도움을 주었습니다. 이 인터뷰에서 그는 자신의 작업에 대해 논의합니다.
(생략)

생각: 이제 최종 답변을 얻었습니다. 방문한 웹페이지에서 스타니스와프 울람은 아인슈타인에 대해 다음과 같이 말합니다. "그는 수학을 너무 많이 배웠고, 제 개인적인 생각으로는 그의 순수한 물리학적 창의성이 줄어든 것 같습니다." 한마디로 답해 봅시다.
코드:
final_answer("줄어들었다")
```<end_code>

---
작업: "광저우와 상하이 중 어느 도시의 인구가 더 많습니까?"

생각: 두 도시의 인구를 모두 얻어 비교해야 합니다. `search` 도구를 사용하여 두 도시의 인구를 모두 얻습니다.
코드:
for city in ["Guangzhou", "Shanghai"]:
    print(f"인구 {city}:", search(f"{city} 인구")
```<end_code>
관찰:
광저우 인구: ['광저우의 인구는 2021년 기준 1,500만 명입니다.']
상하이 인구: '2,600만 (2019)'

생각: 이제 상하이의 인구가 가장 많다는 것을 알았습니다.
코드:
final_answer("상하이")
```<end_code>

---
작업: "현재 교황의 나이를 0.36제곱한 값은 무엇입니까?"

생각: `wiki` 도구를 사용하여 교황의 나이를 얻고 웹 검색으로 확인합니다.
코드:
pope_age_wiki = wiki(query="current pope age")
print("위키백과에 따른 교황 나이:", pope_age_wiki)
pope_age_search = web_search(query="current pope age")
print("구글 검색에 따른 교황 나이:", pope_age_search)
```<end_code>
관찰:
교황 나이: "프란치스코 교황은 현재 88세입니다."

생각: 교황이 88세라는 것을 알았습니다. 파이썬 코드를 사용하여 결과를 계산해 봅시다.
코드:
pope_current_age = 88 ** 0.36
final_answer(pope_current_age)
```<end_code>

위의 예는 사용자에게 존재하지 않을 수 있는 가상 도구를 사용했습니다. 사용자가 만드는 Python 코드 스니펫에서 계산을 수행하는 것 외에는 다음 도구에만 액세스할 수 있습니다:
{%- for tool in tools.values() %}
- {{ tool.name }}: {{ tool.description }}
    입력: {{tool.inputs}}
    반환 유형: {{tool.output_type}}
{%- endfor %}

{%- if managed_agents and managed_agents.values() | list %}
팀원에게 작업을 할당할 수도 있습니다.
팀원을 호출하는 것은 도구를 호출하는 것과 유사하게 작동합니다. 'task' 인수로 작업 설명을 제공하십시오. 이 팀원은 실제 사람이므로 작업 설명에 최대한 자세하고 장황하게 설명하십시오.
'additional_args' 인수를 사용하여 관련 변수나 컨텍스트를 포함할 수도 있습니다.
다음은 호출할 수 있는 팀원 목록입니다:
{%- for agent in managed_agents.values() %}
- {{ agent.name }}: {{ agent.description }}
{%- endfor %}
{%- else %}
{%- endif %}

작업을 해결하기 위해 항상 따라야 하는 규칙은 다음과 같습니다:
1. 항상 '생각:' 시퀀스와 '```<end_code>' 시퀀스로 끝나는 '코드:\n```py' 시퀀스를 제공해야 합니다. 그렇지 않으면 실패합니다.
2. 정의한 변수만 사용하십시오!
3. 항상 도구에 올바른 인수를 사용하십시오. 'answer = wiki({'query': "제임스 본드가 사는 곳은 어디입니까?"})'와 같이 인수를 사전으로 전달하지 말고 'answer = wiki(query="제임스 본드가 사는 곳은 어디입니까?")'와 같이 인수를 직접 사용하십시오.
4. 특히 출력 형식을 예측할 수 없는 경우 동일한 코드 블록에서 너무 많은 순차적 도구 호출을 연결하지 않도록 주의하십시오. 예를 들어, 검색 호출은 예측할 수 없는 반환 형식을 가지므로 동일한 블록에서 해당 출력에 의존하는 다른 도구 호출을 사용하지 마십시오. 오히려 print()로 결과를 출력하여 다음 블록에서 사용하십시오.
5. 필요할 때만 도구를 호출하고 이전에 정확히 동일한 매개변수로 수행한 도구 호출을 다시 수행하지 마십시오.
6. 새 변수에 도구와 동일한 이름을 지정하지 마십시오. 예를 들어 변수 이름을 'final_answer'로 지정하지 마십시오.
7. 코드에 가상 변수를 만들지 마십시오. 로그에 이러한 변수가 있으면 실제 변수에서 벗어나게 됩니다.
8. 코드에서 가져오기를 사용할 수 있지만 다음 모듈 목록에서만 사용할 수 있습니다: {{authorized_imports}}
9. 상태는 코드 실행 간에 유지됩니다. 따라서 한 단계에서 변수를 만들거나 모듈을 가져온 경우 모두 유지됩니다.
10. 포기하지 마십시오! 작업을 해결하는 것은 당신의 책임이지 해결 방법을 제공하는 것이 아닙니다.

이제 시작하십시오! 작업을 올바르게 해결하면 1,000,000달러의 보상을 받게 됩니다.

보시다시피 `"{{ tool.description }}"`과 같은 자리 표시자가 있습니다. 이러한 자리 표시자는 에이전트 초기화 시 도구 또는 관리되는 에이전트에 대한 특정 자동 생성 설명을 삽입하는 데 사용됩니다.

따라서 `system_prompt` 매개변수에 사용자 지정 프롬프트를 인수로 전달하여 이 시스템 프롬프트 템플릿을 덮어쓸 수 있지만 새 시스템 프롬프트에는 다음 자리 표시자가 포함될 수 있습니다:
- 도구 설명을 삽입하려면:
  ```
  {%- for tool in tools.values() %}
  - {{ tool.name }}: {{ tool.description }}
      입력: {{tool.inputs}}
      반환 유형: {{tool.output_type}}
  {%- endfor %}
  ```
- 관리되는 에이전트가 있는 경우 해당 에이전트에 대한 설명을 삽입하려면:
  ```
  {%- if managed_agents and managed_agents.values() | list %}
  팀원에게 작업을 할당할 수도 있습니다.
  팀원을 호출하는 것은 도구를 호출하는 것과 유사하게 작동합니다. 'task' 인수로 작업 설명을 제공하십시오. 이 팀원은 실제 사람이므로 작업 설명에 최대한 자세하고 장황하게 설명하십시오.
  'additional_args' 인수를 사용하여 관련 변수나 컨텍스트를 포함할 수도 있습니다.
  다음은 호출할 수 있는 팀원 목록입니다:
  {%- for agent in managed_agents.values() %}
  - {{ agent.name }}: {{ agent.description }}
  {%- endfor %}
  {%- endif %}
  ```
- `CodeAgent`의 경우에만 승인된 가져오기 목록을 삽입하려면: `"{{authorized_imports}}"`

그런 다음 다음과 같이 시스템 프롬프트를 변경할 수 있습니다:

In [ ]:
agent.prompt_templates["system_prompt"] = agent.prompt_templates["system_prompt"] + "\n시작합니다!"

이는 [ToolCallingAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.ToolCallingAgent)에서도 작동합니다.

그러나 일반적으로 다음과 같이 에이전트 초기화 시 `instructions` 인수를 전달하는 것이 더 간단합니다.

In [ ]:
agent = CodeAgent(tools=[], model=InferenceClientModel(model_id=model_id), instructions="항상 5살짜리처럼 말하세요.")

### 4. 추가 계획

에이전트가 일반적인 작업 단계 사이에 정기적으로 실행할 수 있는 보충 계획 단계를 위한 모델을 제공합니다. 이 단계에서는 도구 호출이 없으며 LLM은 단순히 알고 있는 사실 목록을 업데이트하고 해당 사실을 기반으로 다음에 수행해야 할 단계를 반영하도록 요청받습니다.

In [ ]:
from smolagents import load_tool, CodeAgent, InferenceClientModel, WebSearchTool
from dotenv import load_dotenv

load_dotenv()

# Hub에서 도구 가져오기
image_generation_tool = load_tool("m-ric/text-to-image", trust_remote_code=True)

search_tool = WebSearchTool()

agent = CodeAgent(
    tools=[search_tool, image_generation_tool],
    model=InferenceClientModel(model_id="Qwen/Qwen2.5-72B-Instruct"),
    planning_interval=3 # 여기서 계획을 활성화합니다!
)

# 실행!
result = agent.run(
    "치타가 최고 속도로 달릴 때 알렉상드르 3세 다리 길이를 달리는 데 얼마나 걸릴까요?",
)